# Twitter API Introduction

In this tutorial, we will be praciticing querying the Twitter API in python.

We'll also be using Tweepy - which is a python wrapper that allows us to use the Twitter API. Of course, the Twitter API can be accessed directly as well, but Tweepy has a few extras that makes getting Tweets just a bit easier. 


### Resources and Links

http://docs.tweepy.org/en/latest/


In [ ]:
import json
import networkx as nx
import tweepy
# conda install -c conda-forge tweepy
import numpy as np
import matplotlib.pyplot as plt

### Getting New Tweets

Before you can access the API, you'll need to create a developer account and create/register your application wtih Twitter to get the necessary authentication keys. 

https://developer.twitter.com/

In [ ]:
info = {"api_key": "",
        "api_key_secret": "",
        "bearer_token" : "",
        "access_token": "",
        "access_token_secret": ""}

My app: https://developer.twitter.com/en/portal/projects/1481027098718334976/apps

In [ ]:
info = json.loads(open('../.secrets/twitter_api.json').read())

In [ ]:
# Authenticate using our credentials

client = tweepy.Client(
    consumer_key       = info["api_key"],
    consumer_secret    = info["api_key_secret"],
    bearer_token       = info["bearer_token"],
    access_token       = info["access_token"],
    access_token_secret= info["access_token_secret"],
    wait_on_rate_limit = True
)

`search_all_tweets` - academic research mode only  
https://developer.twitter.com/en/products/twitter-api/academic-research  
https://developer.twitter.com/en/docs/twitter-api/tweets/search/introduction  
https://docs.tweepy.org/en/stable/client.html#search-tweets  

In [ ]:
covid_tweets = client.search_recent_tweets(
    query="covid-19",
    max_results=100,
)

In [ ]:
len(covid_tweets[0])

In [ ]:
type(covid_tweets)

In [ ]:
for tweet in covid_tweets[0]:
    print(type(tweet))
    print(tweet.data)
    break

In [ ]:
len(covid_tweets),covid_tweets[1], covid_tweets[2]

In [ ]:
covid_tweets[3]

In [ ]:
covid_tweets_2 = client.search_recent_tweets(
    query="covid-19",
    max_results=10,
    next_token='b26v89c19zqg8o3fpe77ew01eq35wboat1snf8lq8h5kt'
)

In [ ]:
len(covid_tweets_2[0])

https://developer.twitter.com/en/docs/twitter-api/fields

In [ ]:
fields = "created_at,public_metrics"

In [ ]:
covid_tweets_field = client.search_recent_tweets(
    query="covid",
    max_results=10,
    tweet_fields=fields
)

In [ ]:
for tweet in covid_tweets_field[0]:
    print(tweet.data)
    print(tweet['created_at'])
    break

### Formulating a new query + saving data

https://developer.twitter.com/en/docs/tutorials/building-high-quality-filters

In [ ]:
expansions = "author_id,referenced_tweets.id.author_id"

In [ ]:
tweets_field_exp = client.search_recent_tweets(
    query="#StopAsianHate",
    max_results=10,
    tweet_fields=fields,
    expansions=expansions
)

In [ ]:
### Saving data to your machine
tweet_data_list = [tweet.data for tweet in tweets_field_exp[0]]
with open("../datasets/query1.json", "w") as outfile:
    json.dump(tweet_data_list, outfile, indent=4)


### Getting re-tweeters

https://docs.tweepy.org/en/stable/client.html#tweet-lookup

In [ ]:
tweet_data_list = json.loads(open('./query1.json').read())

In [ ]:
retweeters = []
# Go through the tweet data list
for tweet_dict in tweet_data_list:
    # Get the referenced tweets
    ref_tweet = tweet_dict.get('referenced_tweets')
    # If there is one, get the first one and see if it's a retweet or quote tweet
    if ref_tweet and ref_tweet[0]['type'] in ['retweeted','quoted']:
        # Get that referenced tweet
        ref_tweet = client.get_tweet(
            id=ref_tweet[0]['id'],
            expansions='author_id'
        )
        # add the original and the referenced to the list
        retweeters.append(
            [ tweet_dict.get('author_id'), ref_tweet[0].data.get('author_id')]
        )em

In [ ]:
retweeters

In [ ]:
connections = np.array(retweeters).flatten()

nodes = list(set(connections))

len(connections), len(nodes)

In [ ]:
TW_G = nx.DiGraph()
TW_G.add_nodes_from(nodes)
TW_G.add_edges_from(retweeters)

In [ ]:
ax = plt.subplots()

pos = nx.spring_layout(TW_G, seed=170)
nx.draw_networkx_edges(TW_G, pos, alpha=0.75, width=2)
nx.draw_networkx_nodes(TW_G, pos, node_size=100).set_edgecolor('black')

plt.tight_layout()
plt.axis("off")
plt.show()